In [1]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np
import matplotlib.pyplot as plt
import pickle
from functional.backtester import Backtester as b

In [2]:
start_date = datetime(2020,1,1)
end_date = datetime(2023,1,1)

In [3]:
market = Market()
speculation_db = ADatabase("spec")
classification_db = ADatabase("spec_classification")
class_spec = ADatabase("class_spec")

In [4]:
market.connect()
prices = market.retrieve("prices")
sp500 = market.retrieve("sp500")
market.disconnect()
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [5]:
speculation_db.connect()
simulation = speculation_db.retrieve("sim")
speculation_db.disconnect()

In [6]:
classification_db.connect()
classification_simulation = classification_db.retrieve("sim")
classification_db.disconnect()

In [7]:
simulation["week"] = simulation["week"] + 1
classification_simulation["week"] = classification_simulation["week"] + 1
classification_simulation.rename(columns={"prediction":"classification_prediction"},inplace=True)

In [8]:
simulation = prices.merge(simulation.drop("adjclose",axis=1),on=["year","week","ticker"],how="left") \
                    .merge(classification_simulation.drop(["adjclose","training_years"],axis=1),on=["year","week","ticker"],how="left")

In [9]:
sim = simulation.dropna()

In [10]:
sim["delta"] = (sim["prediction"] - sim["adjclose"]) / sim["adjclose"]

In [11]:
sim.sort_values("date",inplace=True)
sim

,date,close,high,low,open,volume,adjclose,adjhigh,adjlow,adjopen,...,d3,rolling14,xgb_prediction_y,xgb_score_y,cat_prediction_y,cat_score_y,tf_prediction_y,tf_score_y,classification_prediction,delta
4782,2019-01-07,190.88,192.300,188.660,191.36,2162158,163.157226,164.370990,161.259652,163.567513,...,0.574595,168.334661,1.0,0.578265,1.0,0.589024,1.0,0.550765,True,-0.000285
1830432,2019-01-07,22.51,22.630,20.760,21.00,4561940,22.510000,22.630000,20.760000,21.000000,...,9.293771,23.606393,1.0,0.578265,1.0,0.589024,1.0,0.550765,True,-0.053205
861104,2019-01-07,87.59,88.100,83.440,84.89,6611453,86.545407,87.049325,82.444900,83.877607,...,32.948125,90.367242,1.0,0.578265,1.0,0.589024,1.0,0.550765,True,0.026849
1385401,2019-01-07,45.33,45.710,44.010,44.49,6711375,37.269384,37.581812,36.184107,36.578754,...,-4.959443,41.593921,0.0,0.578265,0.0,0.589024,1.0,0.550765,False,-0.103241
1391221,2019-01-07,27.92,28.190,27.250,27.65,8238552,25.509728,25.756420,24.897568,25.263037,...,6.317052,26.423729,1.0,0.578265,1.0,0.589024,1.0,0.550765,True,0.002826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638734,2022-12-30,118.00,118.210,116.010,116.07,4220531,117.466584,117.675635,115.485580,115.545308,...,1.492858,117.924360,1.0,0.584045,1.0,0.589214,1.0,0.566774,True,-0.030499
2019454,2022-12-30,128.99,130.135,128.000,129.88,801280,128.990000,130.135000,128.000000,129.880000,...,15.094060,134.118058,1.0,0.584045,0.0,0.589214,1.0,0.566774,True,0.018259
644554,2022-12-30,95.31,96.750,94.545,96.62,1224392,94.470036,95.897345,93.711778,95.768491,...,0.408714,89.831937,0.0,0.584045,0.0,0.589214,1.0,0.566774,False,0.003871
587089,2022-12-30,140.33,142.550,139.015,142.35,789566,139.165190,141.366763,137.861105,141.168423,...,-0.737267,139.908841,0.0,0.584045,1.0,0.589214,1.0,0.566774,True,0.037576


In [12]:
reqs = [0.05,0.1,0.15]
signals = [0.05,0.1,0.15]
values = [True,False]
classifications = [True,False]
ceilings = [True,False]
floors = [True,False]
hedges = [True,False]
parameters = []
training_year = 4
for value in values:
    for classification in classifications:
        for ceiling in ceilings:
            for floor in floors:
                for hedge in hedges:
                    for signal in signals:
                        for req in reqs:
                            parameter = {"value":value
                                         ,"classification":classification
                                         ,"ceiling":ceiling
                                         ,"floor":floor
                                         ,"hedge":hedge
                                         ,"signal":signal
                                         ,"req":req
                                         ,"training_years":training_year}
                            parameters.append(parameter)

In [13]:
len(parameters)

288

In [14]:
parameters[0]

{'value': True,
 'classification': True,
 'ceiling': True,
 'floor': True,
 'hedge': True,
 'signal': 0.05,
 'req': 0.05,
 'training_years': 4}

In [15]:
class_spec.connect()
class_spec.drop("trades")
for parameter in tqdm(parameters):
    current_sim = sim[sim["training_years"]==parameter["training_years"]].reset_index().copy()
    b.speculation_backtest(current_sim,parameter,start_date,end_date,class_spec)
class_spec.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 288/288 [23:03:12<00:00, 288.17s/it]
